In [2]:
import pandas as pd
from pandas.io import sql

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect('dat-test-solution.db')

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [7]:
data = pd.read_csv('../assets/dataset/rossmann.csv', low_memory=False)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [8]:
data.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)

In [23]:
sql.read_sql('select * from rossmann_sales limit 5', con=conn)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


> #### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [10]:
rossmann_stores = pd.read_csv('../assets/dataset/rossmann-stores.csv')
rossmann_stores.to_sql('rossmann_stores', if_exists='replace', index=False, con=conn)

In [11]:
rossmann_stores.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


> #### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [12]:
sql.read_sql(
"""
SELECT 
Store, Customers 
FROM rossmann_sales;
""", con=conn).head()

,Store,Customers
0,1,555
1,2,625
2,3,821
3,4,1498
4,5,559


> #### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [25]:
sql.read_sql(
"""
SELECT 
store, date, Customers 
FROM rossmann_sales
WHERE Open = 1 and Promo = 1;
""", con=conn).head()

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


> #### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [32]:
sql.read_sql(
"""
SELECT
Promo, Store, SUM(Sales)
FROM rossmann_sales
GROUP BY Promo, Store
""", con=conn).head()

,Promo,Store,SUM(Sales)
0,0,1,1861815
1,0,2,1661644
2,0,3,2309141
3,0,4,3823123
4,0,5,1502882


In [33]:
sql.read_sql(
"""
SELECT
Promo, SUM(Sales)
FROM rossmann_sales
GROUP BY Promo
""", con=conn).head()

,Promo,SUM(Sales)
0,0,2771974337
1,1,3101206286


#### Exercises

1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

In [16]:
# Load the Walmart sales and store features data
walmart_sales = pd.read_csv('../assets/dataset/walmart-sales.csv')
walmart_sales.head()

walmart_features = pd.read_csv('../assets/dataset/features.csv')
walmart_features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [17]:
# Create a table for each of those datasets
walmart_sales.to_sql('walmart_sales', index=False, if_exists='replace', con = conn)
walmart_features.to_sql('walmart_features', if_exists='replace', index=False, con=conn)

In [18]:
# Select the store, date and fuel price on days it was over 90 degrees
query = """
SELECT 
Date, Store, Fuel_Price, Temperature
FROM walmart_features
WHERE Temperature > 90
"""
sql.read_sql(query, con = conn).head()

,Date,Store,Fuel_Price,Temperature
0,2011-08-05,1,3.684,91.65
1,2011-08-12,1,3.638,90.76
2,2011-07-29,2,3.682,90.07
3,2011-08-05,2,3.684,93.34
4,2011-08-12,2,3.638,91.58


In [19]:
# Select the store, date and weekly sales and temperature
query = """
SELECT 
s.Store, s.Date, s.Weekly_Sales, f.Temperature
FROM walmart_sales as s
JOIN walmart_features as f
ON s.Store = f.Store and s.Date = f.Date
"""
sql.read_sql(query, con = conn).head()

,Store,Date,Weekly_Sales,Temperature
0,1,2010-02-05,24924.50,42.31
1,1,2010-02-12,46039.49,38.51
2,1,2010-02-19,41595.55,39.93
3,1,2010-02-26,19403.54,46.63
4,1,2010-03-05,21827.90,46.50


In [34]:
# What were average sales on holiday vs. non-holiday sales
query = """
SELECT 
IsHoliday, AVG(Weekly_Sales) as average
FROM walmart_sales as s
GROUP BY IsHoliday
"""
sql.read_sql(query, con = conn)

,IsHoliday,average
0,0,15901.445069
1,1,17035.823187


In [21]:
# What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees
query = """
SELECT 
s.IsHoliday, AVG(s.Weekly_Sales)
FROM walmart_sales as s
JOIN walmart_features as f
ON s.Store = f.Store and s.Date = f.Date
WHERE f.Temperature < 32
GROUP BY s.IsHoliday
"""
sql.read_sql(query, con = conn)

,IsHoliday,AVG(s.Weekly_Sales)
0,0,15275.770307
1,1,15111.471238
